In [1]:
import pandas as pd
import joblib

In [2]:
#PM 2.5 ESTACIONES INIA Y PUREN
d_pm25_inia = pd.read_csv('../../data/raw/pm2.5_INIA_080913_230807.csv',sep=';', thousands=None, decimal=',')
d_pm25_puren = pd.read_csv('../../data/raw/pm2.5_puren_120315_230808.csv',sep=';', thousands=None, decimal=',')

#PM10 ESTACIONES INIA Y PUREN
d_pm10_inia = pd.read_csv('../../data/raw/pm10_INIA_080101_230807.csv',sep=';', thousands=None, decimal=',')
d_pm10_puren = pd.read_csv('../../data/raw/pm10_puren_120316_230807.csv',sep=';', thousands=None, decimal=',')

In [3]:
#exportamos diccionario
dataframes_dict = {
    'pm_2.5_inia': d_pm25_inia,
    'pm_2.5_puren': d_pm25_puren,
    'pm_10_inia': d_pm10_inia,
    'pm_10_puren': d_pm10_puren
}


for clave, df in dataframes_dict.items():
    # path para el guardado de archivos
    #convertimos FECHA (YYMMDD) a string
    df['FECHA (YYMMDD)'] = df['FECHA (YYMMDD)'].astype(str)

    #rellenamos con 0 para dar uniformidad
    df['FECHA (YYMMDD)'] = df['FECHA (YYMMDD)'].str.zfill(6)

    # eliminamos el campo hora
    df = df.drop('HORA (HHMM)', axis=1)

    #dar formato date
    df['FECHA (YYMMDD)'] = pd.to_datetime(df['FECHA (YYMMDD)'], format='%y%m%d')

    # filtramos solamente registros validados
    df = df[df['Registros validados'].notnull()]

    #eliminamos registros preliminares
    df = df.drop('Registros preliminares', axis=1)

    #eliminamos registros no validados
    df = df.drop('Registros no validados', axis=1)

    #eliminamos columnas 'Unnamed'
    df.drop('Unnamed: 5', axis=1, inplace=True)

    
    # #POR CADA FECHA QUE SE REPITA SE DEJA EL PROMEDIO DE TODOS LOS VALORES DE LA FECHA COMO UNA UNICA TUPLA DEBIDO AL REGISTRO HORARIO DE UN DF EN PARTICULAR
    df = df.groupby('FECHA (YYMMDD)')['Registros validados'].mean().reset_index()


    #renombramos columas
    df = df.rename(columns={
    'FECHA (YYMMDD)': 'fecha',
    'Registros validados': 'registro'
    })


    #HACEMOS EFECTIVO EL CAMBIO
    dataframes_dict[clave] = df

joblib.dump(dataframes_dict, 'estaciones.joblib')


['estaciones.joblib']